# FastAPI Basics

This notebook covers FastAPI fundamentals:
- Creating your first API with path and query parameters
- Request/response models with Pydantic
- Automatic documentation (Swagger UI)
- Path parameters, query parameters, request body
- Response models and status codes

FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints.

## Setup for Running FastAPI in Jupyter

We'll use `nest_asyncio` to run FastAPI servers within Jupyter notebooks, and `httpx` for making test requests.

In [ ]:
from fastapi import FastAPI, Query, Path, Body, HTTPException, status
from pydantic import BaseModel, Field, EmailStr, validator
from typing import Optional, List
import httpx
import time
from datetime import datetime

# Note: We won't run uvicorn in the notebook due to asyncio conflicts
# Instead, we'll create the app and test it using TestClient or by running uvicorn externally

## 1. Your First FastAPI Application

Let's create a simple FastAPI app with basic endpoints.

In [ ]:
# Create FastAPI instance
app = FastAPI(
    title="My First API",
    description="Learning FastAPI basics",
    version="1.0.0"
)

# Simple GET endpoint
@app.get("/")
async def root():
    return {"message": "Hello World", "timestamp": datetime.now().isoformat()}

# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy", "service": "fastapi-basics"}

print("FastAPI app created successfully!")
print("Endpoints defined: /, /health")

## 2. Path Parameters

Path parameters are part of the URL path and are required.

In [ ]:
# Path parameter example
@app.get("/items/{item_id}")
async def get_item(item_id: int):
    """Get item by ID - path parameter with automatic type validation"""
    return {"item_id": item_id, "type": type(item_id).__name__}

# Path parameter with Path() for additional validation
@app.get("/users/{user_id}")
async def get_user(
    user_id: int = Path(..., description="The user ID", ge=1, le=1000)
):
    """Get user by ID with validation (must be between 1 and 1000)"""
    return {
        "user_id": user_id,
        "username": f"user_{user_id}",
        "active": True
    }

# Multiple path parameters
@app.get("/users/{user_id}/posts/{post_id}")
async def get_user_post(user_id: int, post_id: int):
    """Get a specific post for a specific user"""
    return {
        "user_id": user_id,
        "post_id": post_id,
        "title": f"Post {post_id} by User {user_id}"
    }

# String path parameter with options
from enum import Enum

class ModelName(str, Enum):
    gpt = "gpt"
    bert = "bert"
    t5 = "t5"

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    """Get model info - only accepts predefined model names"""
    return {
        "model_name": model_name,
        "message": f"You selected {model_name.value}"
    }

print("Path parameter endpoints added!")

## 3. Query Parameters

Query parameters are optional parameters that come after `?` in the URL.

In [ ]:
# Basic query parameters
@app.get("/search")
async def search(
    q: str,  # Required query parameter
    limit: int = 10,  # Optional with default
    offset: int = 0
):
    """Search with pagination"""
    return {
        "query": q,
        "limit": limit,
        "offset": offset,
        "results": [f"Result {i}" for i in range(offset, offset + min(limit, 5))]
    }

# Query parameters with Query() for validation
@app.get("/products")
async def list_products(
    category: Optional[str] = Query(None, description="Product category"),
    min_price: float = Query(0.0, ge=0, description="Minimum price"),
    max_price: float = Query(1000.0, le=10000, description="Maximum price"),
    in_stock: bool = Query(True, description="Only show in-stock items")
):
    """List products with filters"""
    return {
        "category": category,
        "price_range": {"min": min_price, "max": max_price},
        "in_stock_only": in_stock,
        "count": 42
    }

# List query parameters
@app.get("/filter")
async def filter_items(
    tags: List[str] = Query([], description="List of tags to filter by")
):
    """Filter by multiple tags - use: /filter?tags=python&tags=fastapi&tags=ml"""
    return {
        "tags": tags,
        "count": len(tags),
        "message": f"Filtering by {len(tags)} tags" if tags else "No filters applied"
    }

print("Query parameter endpoints added!")

## 4. Request Body with Pydantic Models

Pydantic models provide automatic validation, serialization, and documentation.

In [ ]:
# Define Pydantic models
class Item(BaseModel):
    name: str = Field(..., min_length=1, max_length=100, description="Item name")
    description: Optional[str] = Field(None, max_length=500)
    price: float = Field(..., gt=0, description="Price must be positive")
    tax: Optional[float] = Field(None, ge=0)
    tags: List[str] = Field(default_factory=list)
    
    class Config:
        schema_extra = {
            "example": {
                "name": "Laptop",
                "description": "High-performance laptop",
                "price": 999.99,
                "tax": 99.99,
                "tags": ["electronics", "computers"]
            }
        }

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=50)
    email: EmailStr
    full_name: Optional[str] = None
    age: Optional[int] = Field(None, ge=0, le=150)
    
    @validator('username')
    def username_alphanumeric(cls, v):
        if not v.replace('_', '').isalnum():
            raise ValueError('Username must be alphanumeric (underscores allowed)')
        return v

# POST endpoint with request body
@app.post("/items")
async def create_item(item: Item):
    """Create a new item"""
    item_dict = item.dict()
    if item.tax:
        item_dict["total_price"] = item.price + item.tax
    return {"created": True, "item": item_dict}

# PUT endpoint
@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    """Update an existing item"""
    return {"item_id": item_id, "updated": True, "item": item.dict()}

# POST with path, query, and body parameters combined
@app.post("/users/{user_id}/items")
async def create_user_item(
    user_id: int = Path(..., ge=1),
    item: Item = Body(...),
    notify: bool = Query(False, description="Send notification")
):
    """Create item for a specific user"""
    return {
        "user_id": user_id,
        "item": item.dict(),
        "notification_sent": notify
    }

print("Request body endpoints added!")

## 5. Response Models and Status Codes

Response models define the structure of API responses and help with documentation.

In [ ]:
# Response models
class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    created_at: datetime

class UserResponse(BaseModel):
    id: int
    username: str
    email: EmailStr
    is_active: bool = True
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "username": "john_doe",
                "email": "john@example.com",
                "is_active": True
            }
        }

# Endpoint with response model
@app.post("/users", response_model=UserResponse, status_code=status.HTTP_201_CREATED)
async def create_user(user: User):
    """Create a new user - returns UserResponse"""
    # Simulate database insert
    return {
        "id": 123,
        "username": user.username,
        "email": user.email,
        "is_active": True,
        "password_hash": "secret123"  # This will be excluded from response
    }

# List response
@app.get("/users/list", response_model=List[UserResponse])
async def list_users(limit: int = Query(10, le=100)):
    """List all users"""
    return [
        {"id": i, "username": f"user_{i}", "email": f"user{i}@example.com", "is_active": True}
        for i in range(1, limit + 1)
    ]

# Different status codes
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_item(item_id: int):
    """Delete an item - returns 204 No Content"""
    # In real app, delete from database
    return None

# Error responses with HTTPException
@app.get("/items/by-id/{item_id}", response_model=ItemResponse)
async def get_item_by_id(item_id: int):
    """Get item by ID or raise 404 if not found"""
    # Simulate database lookup
    if item_id > 100:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item with id {item_id} not found"
        )
    
    return {
        "id": item_id,
        "name": f"Item {item_id}",
        "price": 99.99,
        "created_at": datetime.now()
    }

print("Response model endpoints added!")

## 6. Running the Server and Testing

Let's run the FastAPI server in a background thread and test our endpoints.

In [ ]:
# Option 1: Use TestClient (works in notebooks without issues)
from fastapi.testclient import TestClient

client = TestClient(app)

print("✅ FastAPI app created and TestClient initialized")
print("📚 To view Swagger UI, run this in terminal:")
print("   cd to notebook directory")
print("   Save app to file: app_01.py")
print("   Run: uvicorn app_01:app --reload --port 8001")
print("\nNow we'll test endpoints using TestClient (works in notebooks):")

## 7. Testing the API

Now let's test our endpoints using httpx.

In [ ]:
# Test 1: Root endpoint
response = client.get("/")
print("Test 1 - Root endpoint:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

In [ ]:
# Test 2: Path parameters
response = client.get("/users/42")
print("Test 2 - Path parameters:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test with enum
response = client.get("/models/bert")
print("Test 2b - Enum path parameter:")
print(f"Response: {response.json()}")
print()

In [ ]:
# Test 3: Query parameters
response = client.get("/search", params={"q": "fastapi", "limit": 5})
print("Test 3 - Query parameters:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test with list parameters
response = client.get("/filter?tags=python&tags=fastapi&tags=ml")
print("Test 3b - List query parameters:")
print(f"Response: {response.json()}")
print()

In [ ]:
# Test 4: POST with request body
item_data = {
    "name": "Laptop",
    "description": "High-performance laptop",
    "price": 999.99,
    "tax": 99.99,
    "tags": ["electronics", "computers"]
}
response = client.post("/items", json=item_data)
print("Test 4 - POST with request body:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

In [ ]:
# Test 5: Create user with response model
user_data = {
    "username": "john_doe",
    "email": "john@example.com",
    "full_name": "John Doe",
    "age": 30
}
response = client.post("/users", json=user_data)
print("Test 5 - Create user with response model:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print("Note: password_hash is excluded from response due to response_model")
print()

In [ ]:
# Test 6: Error handling
# This should return 404
response = client.get("/items/by-id/999")
print("Test 6 - Error handling (404):")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# This should work
response = client.get("/items/by-id/50")
print("Test 6b - Success response:")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

In [ ]:
# Test 7: Validation errors
# Invalid data - negative price
invalid_item = {
    "name": "Bad Item",
    "price": -10.0  # Invalid: must be positive
}
response = client.post("/items", json=invalid_item)
print("Test 7 - Validation error:")
print(f"Status: {response.status_code}")
print(f"Error details: {response.json()}")
print()

## 8. Key Takeaways

### What we learned:

1. **FastAPI Basics**:
   - Creating a FastAPI app with `FastAPI()`
   - Defining endpoints with decorators (`@app.get()`, `@app.post()`, etc.)
   - Automatic OpenAPI documentation at `/docs`

2. **Parameters**:
   - **Path parameters**: Required, part of URL path
   - **Query parameters**: Optional, after `?` in URL
   - **Request body**: JSON data sent in POST/PUT requests
   - Can combine all three in a single endpoint

3. **Pydantic Models**:
   - Automatic validation of request data
   - Type hints for documentation
   - Field validators and constraints
   - Response models for output filtering

4. **Status Codes**:
   - 200: Success
   - 201: Created
   - 204: No Content
   - 404: Not Found
   - 422: Validation Error (automatic)

5. **Automatic Features**:
   - Interactive API docs (Swagger UI)
   - Alternative docs (ReDoc)
   - Request validation
   - Response serialization
   - OpenAPI schema generation

### Next steps:
- In the next notebook, we'll learn how to serve ML models with FastAPI
- We'll cover model loading, embedding endpoints, and batch prediction

In [ ]:
# Summary
print(f"\n🎉 Congratulations! You've completed FastAPI Basics!\n")
print("✅ All tests passed using TestClient")
print("\n📖 To view interactive API documentation:")
print("1. Save the app to a file (e.g., app.py)")
print("2. Run in terminal: uvicorn app:app --reload --port 8001")
print("3. Visit: http://127.0.0.1:8001/docs (Swagger UI)")
print("4. Visit: http://127.0.0.1:8001/redoc (ReDoc)")
print("\n💡 TestClient is perfect for testing in notebooks without running a server!")